In [16]:
import torch
import torch.nn as nn
from torchsummary import summary
from torchvision import models


import sys, os
sys.path.append(os.path.abspath("../.."))

%load_ext autoreload
%autoreload 2

from models.res_u_net.model import ResUnetAutoencoder
from models.res_u_net.std_blocks import UpBlock
from models.res_u_net.bridge import Bridge


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
resnet = models.resnet18(pretrained=True)

In [5]:
bridge_out_channels = 1024

In [43]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder_i1, self.bridge_i1, self.decoder_i1 = self._build_enc_dec()
        _, self.bridge_inew, self.decoder_inew = self._build_enc_dec()
        self.encoder_i2, self.bridge_i2, self.decoder_i2 = self._build_enc_dec()

        self.encoder_i1.freeze()
        self.encoder_i2.freeze()


    def _build_enc_dec(self):
        auto_encoder = ResUnetAutoencoder(resnet, bridge_out_channels, UpBlock.UPSAMPLING_BILINEAR)
        return (
            auto_encoder.encoder, auto_encoder.bridge, auto_encoder.decoder
        )
    
    def forward(self, i1, i2):
        ls1 = self.encoder_i1(self.bridge_i1(i1))
        ls2 = self.encoder_i2(self.bridge_i1(i2))

        lsnew = ls1 - ls2

        out1 = self.decoder_i1(ls1 + lsnew)
        out2 = self.decoder_i2(ls2 + lsnew)

        return out1, out2, lsnew
        

In [45]:
net = Net()

In [24]:
i1, i2 = [torch.rand(1, 3, 244, 244) for _ in range(2)]